In [2]:
'''!pip install tensorflow==2.0
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install sentencepiece'''

'!pip install tensorflow==2.0\n!pip install tensorflow_hub\n!pip install bert-for-tf2\n!pip install sentencepiece'

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

TF version:  2.0.0
Hub version:  0.7.0


In [0]:
import bert
from tensorflow.keras.models import Model       # Keras is the new high level API for TensorFlow
from tensorflow import keras
import math
import re
import numpy as np
import random 

In [0]:
checkpoint_path = '/content/model.ckpt'
sentenceFile = '/content/sentences.txt'
treeFile = '/content/tree_mod.txt'

max_seq_length = 128  # Your choice here.
epochs = 1
#textRegEx = "[a-zA-Z1-9!?@:&\+{}\[\]\.,\$/\"-]+"
#tagRegEx = "[A-Z!?@:&\+{}\[\]\.,\$/\"-]+"
#regEx = tagRegEx+"\s"+textRegEx
#print(regEx)
# "$+.[]
nextId = 0
maxVal = 1000
minVal = -1
bertOutDim = 768
#totalTags = 45


In [0]:
label_dict = {'-NONE':1, 'PDT': 2, 'PAP': 3, 'POP': 4, 'A-C': 5, 'CDC': 6, 'IMSAI': 7, 'NP-PRD': 8, 'ADVP-TMP': 9, 'NP-LOC': 10, 'NESB': 11, 'LRB': 12, 'NP-VOC': 13, 'NBC': 14, 'III': 15, 'OK': 16, 'SBAR-LOC-PRD': 17, 'RATE': 18, 'EST': 19, 'MD': 20, 'GHS': 21, 'LANDOR': 22, 'RP': 23, 'C': 24, 'PP-PUT': 25, 'ADVP-LOC-CLR': 26, 'JJS': 27, 'FTC': 28, 'BMP': 29, 'SYM': 30, 'COPPER': 31, 'G': 32, 'LONDON': 33, 'N': 34, 'WTVJ': 35, 'EC': 36, 'ADJP': 37, 'COMMUNICATIONS': 38, 'CS': 39, 'NCR': 40, 'FALL': 41, 'PLO': 42, 'CAT': 43, 'DDB': 44, 'E': 45, 'IBC': 46, 'CIA': 47, 'V': 48, 'FT': 49, 'PLC': 50, 'PRP': 51, 'NX-TTL': 52, 'NNPS': 53, 'BIRDS': 54, 'DT': 55, 'VBZ': 56, 'SEC': 57, 'FUNDS': 58, 'RATES': 59, 'AND': 60, 'ECONOMIC': 61, 'LTV': 62, 'NP-TPC': 63, 'ROGERS': 64, 'FAMILY': 65, 'PP-LOC-CLR-TPC': 66, 'A-D': 67, 'RNR': 68, 'SBARQ-NOM': 69, 'ADR': 70, 'NP-LGS': 71, 'SALARIES': 72, 'ASSOCIATES': 73, 'MONEY': 74, 'LST': 75, 'ADVP-EXT': 76, 'S-SBJ': 77, 'B': 78, 'ADVP-PRD': 79, 'RRC': 80, 'ADVP-PRP': 81, 'WHADJP': 82, 'JUDICIAL': 83, 'PRODUCTS': 84, 'DIALING': 85, 'PP-PRD-LOC': 86, 'OVER': 87, 'ADVP-PRD-LOC': 88, 'FRAG-TPC': 89, 'SQ-TPC': 90, 'SBAR-NOM-SBJ': 91, 'AGREES': 92, 'ADVP-TMP-CLR': 93, 'F': 94, 'OTC': 95, 'FRAG': 96, 'TRIAL': 97, 'LYNCH': 98, 'ADVP-CLR': 99, 'HUD': 100, 'PP-TTL': 101, 'ADJP-TMP': 102, 'LS': 103, 'DES': 104, 'USA': 105, 'US': 106, 'UCP-PRD': 107, 'S-CLF-TPC': 108, 'ASSETS': 109, 'NL': 110, 'JUDGE': 111, 'SBARQ-TPC': 112, 'SBAR': 113, 'S-PRP-PRD': 114, 'NASD': 115, 'NP-TMP': 116, 'MORTGAGE': 117, 'ABA': 118, 'CONJP': 119, 'CHANGED': 120, 'VBD': 121, 'AT': 122, 'ADVP': 123, 'TRANSPLANT': 124, 'PP-CLR': 125, 'PP-BNF': 126, 'VB': 127, 'GRAINS': 128, 'POTABLES': 129, 'UAL': 130, 'WRB': 131, 'THREE': 132, 'ABORTION': 133, 'NOT': 134, 'PP-TMP-PRD': 135, 'WHPP': 136, 'IBM': 137, 'S-TTL': 138, 'IRA': 139, 'NP-HLN': 140, 'FRAG-TTL-SBJ': 141, 'NEC': 142, 'CBS': 143, 'NBI': 144, 'ADVP-MNR': 145, 'A': 146, 'DOT': 147, 'SINV-TPC': 148, 'UBS': 149, 'SALT': 150, 'PP-DIR': 151, 'CD': 152, 'PPA': 153, 'S-TMP': 154, 'ENERGY': 155, 'SERVICES': 156, 'SAT': 157, 'ADJP-PRD': 158, 'TEXAS': 159, 'TWO': 160, 'MEDICINE': 161, 'SBAR-CLR': 162, 'FAX': 163, 'PORTING': 164, 'PS': 165, 'SBAR-MNR': 166, 'FRAG-HLN': 167, 'UCP': 168, 'SBAR-TMP': 169, 'USIA': 170, 'BALLOT': 171, 'MERRILL': 172, 'GROWTH': 173, 'S-MNR': 174, 'OSHA': 175, 'WP': 176, 'FW': 177, 'BANKERS': 178, 'PP-PRP': 179, 'ADVP-LOC': 180, 'TRS': 181, 'PP-CLR-LOC': 182, 'EXP': 183, 'INS': 184, 'S-PRP-CLR': 185, 'INTER-TEL': 186, 'SWITCHING': 187, 'UNION': 188, 'PAPER': 189, 'INTJ': 190, 'ORTEGA': 191, 'LOAN': 192, 'DD': 193, 'AG': 194, 'UH': 195, 'RULING': 196, 'METALS': 197, 'PP-EXT': 198, 'ADJP-PRD-TPC': 199, 'NP-CLR': 200, 'WAR': 201, 'II': 202, 'SINV': 203, 'VBG': 204, 'SBAR-LOC': 205, 'CEO': 206, 'CLEARS': 207, 'IRS': 208, 'GOODY': 209, 'NAC': 210, 'USX': 211, 'TRIMMING': 212, 'TREASURY': 213, 'CSV': 214, 'X': 215, 'AN': 216, 'LSI': 217, 'DISCOUNT': 218, 'SBAR-PRD': 219, 'WHNP': 220, 'IX': 221, 'COMPUTERS': 222, 'PP-LOC-CLR': 223, 'GM': 224, 'NATIONAL': 225, 'ADVP-DIR': 226, 'YWCA': 227, 'CC': 228, 'NP-MNR': 229, 'OF': 230, 'VP-TPC': 231, 'EVERYONE': 232, 'H': 233, 'S-NOM-PRD': 234, 'TIRED': 235, 'ADVP-PRD-TPC': 236, 'X-HLN': 237, 'COLLECTING': 238, 'WPP': 239, 'S-HLN': 240, 'INQUIRY': 241, 'RCB': 242, 'NNP': 243, 'SBAR-NOM-PRD': 244, 'PP-LOC-TPC': 245, 'WAFA': 246, 'RB': 247, 'POS': 248, 'FIRST': 249, 'K': 250, 'NNS': 251, 'NP-TMP-HLN': 252, 'JJR': 253, 'LIBOR': 254, 'FERC': 255, 'TO': 256, 'WHADVP': 257, 'ASSOCIATION': 258, 'WTD': 259, 'DSM': 260, 'NAC-LOC': 261, 'INTERBANK': 262, 'SBAR-ADV': 263, 'PP-TPC': 264, 'LCB': 265, 'TV': 266, 'S-CLF': 267, 'CALL': 268, 'IOU': 269, 'PP-TMP': 270, 'ADJP-CLR': 271, 'Q': 272, 'FRAG-ADV': 273, 'S-NOM': 274, 'RBS': 275, 'SDI': 276, 'EURODOLLARS': 277, 'SHAREDATA': 278, 'SOYBEANS': 279, 'LATE': 280, 'NEW': 281, 'S-TPC': 282, 'NONE': 283, 'ACQUISITION': 284, 'RRB': 285, 'ISSUES': 286, 'PP-MNR': 287, 'ON': 288, 'BTR': 289, 'DIAPER': 290, 'S-CLR': 291, 'NP-EXT': 292, 'CAMPAIGN': 293, 'ASLACTON': 294, 'PP-LGS': 295, 'PP-TMP-CLR': 296, 'NTG': 297, 'NP-TMP-CLR': 298, 'PP-PRD': 299, 'YMCA': 300, 'APPEARS': 301, 'P': 302, 'READY': 303, 'W': 304, 'PRIME': 305, 'BILLS': 306, 'S-ADV': 307, 'Z': 308, 'SCI': 309, 'U': 310, 'PRT': 311, 'ENDED': 312, 'D': 313, 'TROUBLES': 314, 'ADVP-LOC-PRD': 315, 'INTERPUBLIC': 316, 'PP-DTV': 317, 'FOREIGN': 318, 'PHOTOGRAPH': 319, 'NP-SBJ': 320, 'ACCOUNT': 321, 'UPHELD': 322, 'PP-LOC-PRD': 323, 'FBI': 324, 'NAC-TMP': 325, 'ADVP-LOC-PRD-TPC': 326, 'ADVP-DIR-TPC': 327, 'MITI': 328, 'S-PRP': 329, 'CORP': 330, 'SBARQ': 331, 'NCNB': 332, 'NP': 333, 'GMAC': 334, 'ACCEPTANCES': 335, 'PP': 336, 'NP-TTL-PRD': 337, 'IT': 338, 'SBAR-PRP': 339, 'PP-DIR-CLR': 340, 'ADJP-ADV': 341, 'VP': 342, 'DOONESBURY': 343, 'VBN': 344, 'NP-TMP-TPC': 345, 'JJ': 346, 'OFFERED': 347, 'ADVP-HLN': 348, 'EX': 349, 'M': 350, 'HOME': 351, 'FEDERAL': 352, 'PRECIOUS': 353, 'UCP-MNR': 354, 'BRAMALEA': 355, 'NP-TTL': 356, 'NN': 357, 'CTB': 358, 'COMMERCIAL': 359, 'GHKM': 360, 'FT-SE': 361, 'NP-PRD-TTL': 362, 'PRN': 363, 'NIH': 364, 'VOA': 365, 'BRIEFS': 366, 'THE': 367, 'S-PRD': 368, 'EXCHANGE': 369, 'NP-ADV': 370, 'PC': 371, 'ICH': 372, 'DEFENSE': 373, 'J': 374, 'SQ': 375, 'SBAR-PRP-PRD': 376, 'O': 377, 'CERTIFICATES': 378, 'CREATOR': 379, 'ADJP-TPC': 380, 'PP-LOC-PRD-TPC': 381, 'SBAR-SBJ': 382, 'ADVP-PUT': 383, 'NYSE': 384, 'NRDC': 385, 'WFRR': 386, 'AC': 387, 'PP-LOC': 388, 'THAT': 389, 'TXO': 390, 'GAF': 391, 'WHAS': 392, 'HHS': 393, 'QP': 394, 'RBR': 395, 'S-NOM-SBJ': 396, 'UCP-PRP': 397, 'DNA': 398, 'AMR': 399, 'UCP-ADV': 400, 'UCP-TMP': 401, 'PETS': 402, 'WDT': 403, 'ADVP-DIR-CLR': 404, 'DEPOSIT': 405, 'T': 406, 'PP-MNR-CLR': 407, 'VBP': 408, 'EEOC': 409, 'L': 410, 'AIDS': 411, 'R': 412, 'IN': 413, 'PTA': 414, 'GOP': 415, 'UCP-LOC-CLR': 416, 'TRUST': 417, 'INGERSOLL-RAND': 418, 'PAPERS': 419, 'RMS': 420, 'NX': 421, 'RBC': 422, 'I': 423, 'Y': 424, 'ITC': 425, 'S': 426, 'NP-TTL-SBJ': 427, 'CTBS': 428, 'SBAR-NOM': 429,'.':430, '!':431, '?':432, '@':433,':':434,'&':435,'+':436,'{':437,'}':438,'[':439,']':440,'$':441,'/':442,'"':443,'-':444,',':445,'``':446,"''":447,'#':448,'ADVP|PRT':449}

In [7]:
totalTags = len(label_dict)
print(totalTags)

449


In [0]:
def getBERTModel():
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,name="segment_ids")

    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=[pooled_output, sequence_output])

    # Import tokenizer using the original vocab file
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    FullTokenizer = bert.bert_tokenization.FullTokenizer
    tokenizer = FullTokenizer(vocab_file, do_lower_case)

    return {'model':model,'tokenizer':tokenizer}

In [0]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    #print('len(tokens),max_seq_length)
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

In [0]:
def getEmbeddings(model,tokenizer,sentence): 
    stokens = tokenizer.tokenize(sentence)
    #print('sen len:',sentence)
    #print('sen len:',len(sentence.split()))
    stokens = ["[CLS]"] + stokens + ["[SEP]"]

    input_ids = get_ids(stokens, tokenizer, max_seq_length)
    input_masks = get_masks(stokens, max_seq_length)
    input_segments = get_segments(stokens, max_seq_length)

    pool_embs, all_embs = model.predict([[input_ids],[input_masks],[input_segments]])
    # pool_ebmbs is an embeding of CLS token
    # all_embs contains embeding for words of input sentence.
    return all_embs[0]

In [0]:
class Node:
    def __init__(self,lb):
        self.label = lb
        self.I = maxVal
        self.J = minVal
        self.child = []

In [0]:
class stackEle:
    def __init__(self,st,nd):
        self.tag = st
        self.nd = nd

In [0]:
def top(stack):
    return stack[len(stack)-1]

In [14]:
'''def getLeafIndices(regEx,tree):
    iter = re.finditer(regEx, tree)
    indices = [m.start(0) for m in iter]
    return indices'''

'def getLeafIndices(regEx,tree):\n    iter = re.finditer(regEx, tree)\n    indices = [m.start(0) for m in iter]\n    return indices'

In [0]:
def getLabel(tree,treeLen,start):
    label = ''
    i = start
    while(i<treeLen and tree[i] != ' ' and tree[i]!=')'):
        label = label+tree[i]
        i = i + 1
    return [label,i]

In [0]:
def labelToid(label=None, label_dict=label_dict):
    if label is not None:
        raw_label = re.search('(.*?)[-0-9]*$', label).group(1)
        n = len(raw_label)

        if n>1 and raw_label[n-1] == '=':
            raw_label = raw_label[:n-1]
        elif n>1 and raw_label[n-1] == '$':
            raw_label = raw_label[:n-1]
        elif n>1 and raw_label[0] == '-':
            raw_label = raw_label[1:]

        return label_dict[raw_label]
    else:
        return None

In [17]:
'''def generateChild(tree,treeLen,i,head,indices,mat):
    global nextId
    out = getLabel(tree,treeLen,i)
    #print(out)
    temp = Node(out[0])
    if i in indices:
        temp.I = nextId
        temp.J = nextId
        nextId = nextId + 1
        while(i<treeLen and tree[i] != ')'):
            i = i + 1
        i = i + 1
        mat[temp.I][temp.J] = labelToid(temp.label,label_dict)      #------------------------------
        #mat[temp.I][temp.J] = 40
        head.child.append(temp)
        return i
    else: 
        i = out[1]
        while tree[i] == '(':
            ind = generateChild(tree,treeLen,i+1,temp,indices,mat)
            i = ind
            newChild = temp.child[len(temp.child)-1]
            #print(temp.I,newChild.I)
            temp.I = min(temp.I,newChild.I)
            temp.J = max(temp.J,newChild.J)
        i = i + 1
        mat[temp.I][temp.J] = labelToid(temp.label,label_dict) #-------------------------------
        #mat[temp.I][temp.J] = 40
        head.child.append(temp)
        return i'''

"def generateChild(tree,treeLen,i,head,indices,mat):\n    global nextId\n    out = getLabel(tree,treeLen,i)\n    #print(out)\n    temp = Node(out[0])\n    if i in indices:\n        temp.I = nextId\n        temp.J = nextId\n        nextId = nextId + 1\n        while(i<treeLen and tree[i] != ')'):\n            i = i + 1\n        i = i + 1\n        mat[temp.I][temp.J] = labelToid(temp.label,label_dict)      #------------------------------\n        #mat[temp.I][temp.J] = 40\n        head.child.append(temp)\n        return i\n    else: \n        i = out[1]\n        while tree[i] == '(':\n            ind = generateChild(tree,treeLen,i+1,temp,indices,mat)\n            i = ind\n            newChild = temp.child[len(temp.child)-1]\n            #print(temp.I,newChild.I)\n            temp.I = min(temp.I,newChild.I)\n            temp.J = max(temp.J,newChild.J)\n        i = i + 1\n        mat[temp.I][temp.J] = labelToid(temp.label,label_dict) #-------------------------------\n        #mat[temp.I][t

In [18]:
'''def generateTree(tree,mat):
    #print(tree)
    global nextId
    treeLen = len(tree)
    indices = getLeafIndices(regEx,tree)
    i = 0
    i = i + 1
    nextId = 0
    #head = None
    while(i<treeLen):
        out = getLabel(tree,treeLen,i)
        #print(out)
        head = Node(out[0])
        i = out[1]
        while tree[i] == '(':
            ind = generateChild(tree,treeLen,i+1,head,indices,mat)
            i = ind
            newChild = head.child[len(head.child)-1]
            #print(temp.I,newChild.I)
            head.I = min(head.I,newChild.I)
            head.J = max(head.J,newChild.J)
        i = i + 1
        mat[head.I][head.J] = labelToid(head.label,label_dict)    #-----------------------
        #mat[head.I][head.J] = 40'''

"def generateTree(tree,mat):\n    #print(tree)\n    global nextId\n    treeLen = len(tree)\n    indices = getLeafIndices(regEx,tree)\n    i = 0\n    i = i + 1\n    nextId = 0\n    #head = None\n    while(i<treeLen):\n        out = getLabel(tree,treeLen,i)\n        #print(out)\n        head = Node(out[0])\n        i = out[1]\n        while tree[i] == '(':\n            ind = generateChild(tree,treeLen,i+1,head,indices,mat)\n            i = ind\n            newChild = head.child[len(head.child)-1]\n            #print(temp.I,newChild.I)\n            head.I = min(head.I,newChild.I)\n            head.J = max(head.J,newChild.J)\n        i = i + 1\n        mat[head.I][head.J] = labelToid(head.label,label_dict)    #-----------------------\n        #mat[head.I][head.J] = 40"

In [0]:
def generateTree(tree,mat):
    global nextId
    
    nextId = 0
    stack = []
    treeLen = len(tree)
    i = 1
    
    label,ind = getLabel(tree,treeLen,i)
    i = ind
    ele = stackEle(label,Node(label))
    stack.append(ele)

    while i<treeLen-1:
        #print(i)
        if tree[i] == '(':
            stack.append(stackEle('(',None))
            i = i + 1
        elif tree[i] == ')':
            temp = []
            while top(stack).tag != '(':
                tp = top(stack)
                temp.append(tp)
                stack.pop()
            stack.pop()
            if len(temp) == 1:
                tp = top(stack).nd
                newChild = temp[0].nd
                mat[newChild.I][newChild.J] = labelToid(newChild.label,label_dict)      #--------------------------
                tp.I = min(tp.I,newChild.I)
                tp.J = max(tp.J,newChild.J)
                tp.child.append(newChild)
            else:
                tp = top(stack).nd
                newChild = temp[1].nd
                newChild.I = nextId
                newChild.J = nextId
                mat[newChild.I][newChild.J] = labelToid(newChild.label,label_dict)      #--------------------------
                nextId = nextId + 1
                tp.I = min(tp.I,newChild.I)
                tp.J = max(tp.J,newChild.J)
                tp.child.append(newChild)
            i = i + 1
        elif tree[i] == ' ':
            i = i + 1
        else:
            label,ind = getLabel(tree,treeLen,i)
            i = ind
            ele = stackEle(label,Node(label))
            stack.append(ele)

    tp = top(stack).nd
    stack.pop()
    mat[tp.I][tp.J] = labelToid(tp.label,label_dict)     #--------------------------

In [0]:
def genSpanEncode(embs,i,j):
    vi = embs[i]
    vj = embs[j]

    fi = vi[::2]
    bi = vi[1::2]
    fj = vj[::2]
    bj = vj[1::2]

    spanEncode = np.concatenate(((fi - fj), (bi - bj)), axis=0)
    #print(spanEncode.shape)
    return spanEncode

In [0]:
def getGroundTruth(id):
    gt = [0]*(id-1) + [1] + [0]*(totalTags-id)
    #print(len(gt))
    return np.asarray(gt)

In [0]:
def getSTrData(model,tokenizer,sentence,tree):
    length = len(sentence.split())
    mat = [[0 for i in range(length)] for i in range(length)]
    generateTree(tree,mat)
    #print(mat)
    embs = getEmbeddings(model,tokenizer,sentence)

    trainX = np.asarray([])
    trainY = np.asarray([])
    for i in range(length):
        for j in range(i,length):
            spanij = genSpanEncode(embs,i,j)
            #----- It will remove all 'UNK' tag.----------------
            if mat[i][j] == 0:
              '''num = random.randrange(0, 100,1)
              if num > 25:'''
              continue
            #-----------------------------------------------------------------------
            GTij = getGroundTruth(mat[i][j])
            if trainX.shape[0] == 0:
                trainX = np.asarray([spanij])
                trainY = np.asarray([GTij])
            else:
                trainX = np.append(trainX,[spanij],axis=0)
                trainY = np.append(trainY,[GTij],axis=0)
    return trainX,trainY

In [0]:
def getTrainingData(model,tokenizer,nSentences,nTrees):
    trainX = np.asarray([])
    trainY = np.asarray([])
    n = len(nSentences)
    for i in range(n):             #--------------------------------------
        if i%100 == 0:
          print( 'Processing ',i,' out of ',n)
        senLen = len(nSentences[i].split())
        if senLen > 60:
          print('Skipped...')
          continue
        x,y = getSTrData(model,tokenizer,nSentences[i],nTrees[i])
        if trainX.shape[0] == 0:
            trainX = x
            trainY = y
        else:
            trainX = np.concatenate((trainX, x), axis=0)
            trainY = np.concatenate((trainY, y), axis=0)
    return trainX,trainY

In [0]:
def createModel():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(256,input_dim=bertOutDim,activation="relu"))
    model.add(keras.layers.Dense(totalTags,activation="softmax"))
    model.compile(loss="categorical_crossentropy",optimizer="adam",matrics=["accuracy"])
    return model

In [0]:
outModel = getBERTModel()
model = outModel['model']
tokenizer = outModel['tokenizer']

In [0]:
'''nSentence = ["short cuts make long delays","short cuts make long delays"]
nTree = ['(S (NP (JJ short)(NNS cuts))(VP (VBP make)(NP (JJ long)(NNS delays))))',
         '(S (NP (A-C short)(NNS cuts))(VP (VBP make)(NP (JJ long)(NNS delays))))'
         ]'''

In [27]:
f = open(sentenceFile, 'r')
nSentence = f.read().splitlines()
f.close()

"f = open(sentenceFile, 'r')\nnSentence = f.read().splitlines()\nf.close()"

In [28]:
f = open(treeFile, 'r')
nTree = f.read().splitlines()
f.close()

"f = open(treeFile, 'r')\nnTree = f.read().splitlines()\nf.close()"

In [29]:
trainX,trainY = getTrainingData(model,tokenizer,nSentence,nTree)
print('Training data generated.')
print(trainX.shape)
print(trainY.shape)

Processing  0  out of  2
Training data generated.
(18, 768)
(18, 449)


In [0]:
#print(trainY)

In [31]:
np.savetxt('trainX.csv',trainX, delimiter=',',fmt='%8.4f')
np.savetxt('trainY.csv',trainY, delimiter=',',fmt='%8.4f')

"np.savetxt('trainX.csv',trainX, delimiter=',',fmt='%8.4f')\nnp.savetxt('trainY.csv',trainY, delimiter=',',fmt='%8.4f')"

In [34]:
model = createModel()
#model.load_weights(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)
model.fit(trainX,trainY,epochs=epochs,callbacks=[cp_callback])

Train on 18 samples

Epoch 00001: saving model to /content/model.ckpt
18/18 [==============================] - 0s 26ms/sample - loss: 6.2203


In [33]:
'''
model.evaluate(X_test, y_test)

model = createModel()
model.load_weights(checkpoint_path)
y_proba = model.predict(X_new)
'''

'\nmodel.evaluate(X_test, y_test)\n\nmodel = createModel()\nmodel.load_weights(checkpoint_path)\ny_proba = model.predict(X_new)\n'